# Data Preparation

## Fetch Data from CSV

In [1]:
%pip install numpy pandas

  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl (14.9 MB)
                                              0.0/10.8 MB ? eta -:--:--
                                              0.1/10.8 MB 1.1 MB/s eta 0:00:10
                                              0.2/10.8 MB 1.4 MB/s eta 0:00:08
                                              0.2/10.8 MB 1.5 MB/s eta 0:00:07
     -                                        0.3/10.8 MB 1.6 MB/s eta 0:00:07
     -                                        0.4/10.8 MB 1.5 MB/s eta 0:00:07
     -                                        0.5/10.8 MB 1.6 MB/s eta 0:00:07
     --                                       0.6/10.8 MB 1.6 MB/s eta 0:00:07
     --                                       0.6/10.8 MB 1.5 MB/s eta 0:00:07
     --                                       0.7/10.8 MB 1.6 MB/s eta 0:00:07
     --                                       0.8/10.8 MB 1.6 MB/s eta 0:00:07
     ---                                      0.9/10.8 MB 1.6 MB/s eta 0:00:


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd

In [3]:
question_path = './data/dataset-question-v2.csv'
answer_path = './data/dataset-answer.csv'

question_attr = ['pattern', 'tag']
answer_attr = ['responses', 'tag']

question_data = pd.read_csv(question_path)[question_attr]
answer_data = pd.read_csv(answer_path)[answer_attr]

# data_answer.head(10)
print("Jumlah pertanyaan: ", question_data.shape)
print("Jumlah pertanyaan: ", answer_data.shape)

Jumlah pertanyaan:  (558, 2)
Jumlah pertanyaan:  (17, 2)


In [4]:
question_data.head(10)

,pattern,tag
0,Hi,gr_hi
1,Halo,gr_ha
2,Selamat Pagi,gr_pa
3,Selamat Siang,gr_si
4,Selamat Sore,gr_so
5,Selamat Malam,gr_ma
6,Ada program studi apa saja di UIN Sunan Gunung...,prodi
7,Apa saja program studi yang ada di UIN Sunan G...,prodi
8,Program studi apa yang tersedia di UIN Sunan G...,prodi
9,"Di UIN Sunan Gunung Djati Bandung, ada program...",prodi


In [5]:
answer_data.head(10)

,responses,tag
0,Halo! Apa yang ingin anda cari tahu hari ini?,gr_hi
1,Hi! Apa yang bisa kami bantu?,gr_ha
2,Selamat Pagi! Apa yang bisa kami bantu untuk m...,gr_pa
3,Selamat Siang! Apa yang ingin anda cari tahu?,gr_si
4,Selamat Sore! Apa yang ingin anda ketahui?,gr_so
5,Selamat Malam! Apa yang bisa kami bantu?,gr_ma
6,UIN Sunan Gunung Djati Bandung menyelenggaraka...,prodi
7,SNBP adalah singkatan dari Seleksi Nasional Be...,jm_s1
8,Untuk jurusan yang bisa dipilih pada Seleksi M...,snbp_jur
9,Untuk jurusan yang bisa dipilih pada Seleksi M...,snbt_jur


## Data Cleaning

In [8]:
%pip install transformers

  Using cached transformers-4.44.0-py3-none-any.whl (9.5 MB)
  Using cached filelock-3.15.4-py3-none-any.whl (16 kB)
  Using cached huggingface_hub-0.24.5-py3-none-any.whl (417 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached tokenizers-0.19.1-cp38-none-win_amd64.whl (2.2 MB)
  Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB)
  Using cached charset_normalizer-3.3.2-cp38-cp38-win_amd64.whl (99 kB)
  Using cached idna-3.7-py3-none-any.whl (66 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
%pip install tensorflow

  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl (1.9 kB)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorflow-2.13.0-cp38-cp38-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.13.0-cp38-cp38-win_amd64.whl (276.5 MB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.11.0-cp38-cp38-win_amd64.whl (3.0 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached grpcio-1.65.4-cp38-cp38-win_amd64.whl (4.1 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached google_auth-2.33.0-py2.py3-none-any.whl (200 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl 


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from transformers import AutoTokenizer
import re
import tensorflow as tf

In [12]:
SBERT_TOKENIZER = 'firqaaa/indo-sentence-bert-base'

class Preprocess():
    def __init__(self, max_len=128):
        self.stemmer = StemmerFactory().create_stemmer()
        self.stopword = StopWordRemoverFactory().create_stop_word_remover()
        self.tokenizer = AutoTokenizer.from_pretrained(SBERT_TOKENIZER)
        self.max_len = max_len

    def cleaning(self, val):
        val = re.sub(r'\s+', ' ', val)
        val = re.sub("[^a-zA-Z0-9;]", " ", val)
        return val
    
    def casefolding(self, val):
        return str(val).lower()
    
    def stemming(self, val):
        return self.stemmer.stem(str(val))
    
    def stopwordRemoval(self, val):
        return self.stopword.remove(str(val))
    
    def embedding(self, val):
        return self.tokenizer.encode_plus(
            val,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='tf'
        )
    
    def preprocessing(self, sentences):
        for i in range(len(sentences)):
            input = sentences[i]
            input = self.cleaning(input)
            input = self.casefolding(input)
            input = self.stemming(input)
            input = self.stopwordremove(input)
            sentences[i] = input
        return sentences

    def tokenizing(self, sentences):
        input_ids, attention_mask = [], []
        for sentence in sentences:
            output = self.embedding(sentence)
            input_ids.append(output['input_ids'])
            attention_mask.append(output['attention_mask'])
        return {
            'input_ids': tf.convert_to_tensor(
                np.asarray(input_ids).squeeze(),
                dtype=tf.int32
            ),
            'attention_mask': tf.convert_to_tensor(
                np.asarray(attention_mask).squeeze(),
                dtype=tf.int32
            )
        }

    def preprocess_get_token(self, sentences, display_len=20):
        preprocessing = self.preprocessing(sentences)
        tokenized = self.tokenizing(preprocessing)
        return [self.tokenizer.convert_ids_to_tokens(tokenized['input_ids'][i][:display_len]) for i in range(len(sentences))]

preprocess = Preprocess()    

c:\Users\Haikal\AppData\Local\Programs\Python\Python38\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Haikal\.cache\huggingface\hub\models--firqaaa--indo-sentence-bert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Haikal\AppData\Local\Programs\Python\Python38\lib\site-packages\tran

### Casefolding

In [13]:
sample_texts = question_data['pattern']
print(sample_texts[7])
print(sample_texts[8])
print(sample_texts[9])

Apa saja program studi yang ada di UIN Sunan Gunung Djati Bandung?
Program studi apa yang tersedia di UIN Sunan Gunung Djati Bandung?
Di UIN Sunan Gunung Djati Bandung, ada program studi apa saja?


In [14]:
preprocess = Preprocess()
hasil_casefolding = [preprocess.casefolding(sample) for sample in sample_texts]

bf_cf = sample_texts
af_cf = hasil_casefolding

df = pd.DataFrame({
    'Before Casefolding': bf_cf,
    'After Casefolding': af_cf 
})

df.tail(10)

,Before Casefolding,After Casefolding
548,Program beasiswa di UIN Sunan Gunung Djati Ban...,program beasiswa di uin sunan gunung djati ban...
549,Apakah beasiswa di UIN Sunan Gunung Djati Band...,apakah beasiswa di uin sunan gunung djati band...
550,Program beasiswa di UIN Sunan Gunung Djati Ban...,program beasiswa di uin sunan gunung djati ban...
551,Apakah program beasiswa yang ada di UIN Sunan ...,apakah program beasiswa yang ada di uin sunan ...
552,Program beasiswa yang ditawarkan oleh UIN Suna...,program beasiswa yang ditawarkan oleh uin suna...
553,Apakah program beasiswa disediakan oleh UIN Su...,apakah program beasiswa disediakan oleh uin su...
554,Program beasiswa di UIN Sunan Gunung Djati Ban...,program beasiswa di uin sunan gunung djati ban...
555,Apakah beasiswa di UIN Sunan Gunung Djati Band...,apakah beasiswa di uin sunan gunung djati band...
556,Program beasiswa di UIN Sunan Gunung Djati Ban...,program beasiswa di uin sunan gunung djati ban...
557,Apakah beasiswa di UIN Sunan Gunung Djati Band...,apakah beasiswa di uin sunan gunung djati band...


### Cleaning

In [15]:
preprocess = Preprocess()
hasil_cleaning = [preprocess.cleaning(sample) for sample in hasil_casefolding]
bf_cl = hasil_casefolding
af_cl = hasil_cleaning
df = pd.DataFrame({
    'Before Cleaning': bf_cl,
    'After Cleaning': af_cl
})
df.tail(10)

c:\Users\Haikal\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,Before Cleaning,After Cleaning
548,program beasiswa di uin sunan gunung djati ban...,program beasiswa di uin sunan gunung djati ban...
549,apakah beasiswa di uin sunan gunung djati band...,apakah beasiswa di uin sunan gunung djati band...
550,program beasiswa di uin sunan gunung djati ban...,program beasiswa di uin sunan gunung djati ban...
551,apakah program beasiswa yang ada di uin sunan ...,apakah program beasiswa yang ada di uin sunan ...
552,program beasiswa yang ditawarkan oleh uin suna...,program beasiswa yang ditawarkan oleh uin suna...
553,apakah program beasiswa disediakan oleh uin su...,apakah program beasiswa disediakan oleh uin su...
554,program beasiswa di uin sunan gunung djati ban...,program beasiswa di uin sunan gunung djati ban...
555,apakah beasiswa di uin sunan gunung djati band...,apakah beasiswa di uin sunan gunung djati band...
556,program beasiswa di uin sunan gunung djati ban...,program beasiswa di uin sunan gunung djati ban...
557,apakah beasiswa di uin sunan gunung djati band...,apakah beasiswa di uin sunan gunung djati band...


### Stemming

In [16]:
preprocess = Preprocess()
hasil_stemming = [preprocess.stemming(sample) for sample in hasil_cleaning]
bf_st = hasil_cleaning
af_st = hasil_stemming
df = pd.DataFrame({
    "Before Stemming" : bf_st,
    "After Stemming" : af_st
})
df.tail(10)

c:\Users\Haikal\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,Before Stemming,After Stemming
548,program beasiswa di uin sunan gunung djati ban...,program beasiswa di uin sunan gunung djati ban...
549,apakah beasiswa di uin sunan gunung djati band...,apakah beasiswa di uin sunan gunung djati band...
550,program beasiswa di uin sunan gunung djati ban...,program beasiswa di uin sunan gunung djati ban...
551,apakah program beasiswa yang ada di uin sunan ...,apakah program beasiswa yang ada di uin sunan ...
552,program beasiswa yang ditawarkan oleh uin suna...,program beasiswa yang tawar oleh uin sunan gun...
553,apakah program beasiswa disediakan oleh uin su...,apakah program beasiswa sedia oleh uin sunan g...
554,program beasiswa di uin sunan gunung djati ban...,program beasiswa di uin sunan gunung djati ban...
555,apakah beasiswa di uin sunan gunung djati band...,apakah beasiswa di uin sunan gunung djati band...
556,program beasiswa di uin sunan gunung djati ban...,program beasiswa di uin sunan gunung djati ban...
557,apakah beasiswa di uin sunan gunung djati band...,apakah beasiswa di uin sunan gunung djati band...


### Stopword Removal

In [17]:
preprocess = Preprocess()
hasil_stopword_removal = [preprocess.stopwordRemoval(sample) for sample in hasil_stemming]
bf_sr = hasil_stemming
af_sr = hasil_stopword_removal
df = pd.DataFrame({
    "Before Stopword Removal": bf_sr,
    "After Stopword Removal": af_sr
})
df.tail(10)

c:\Users\Haikal\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,Before Stopword Removal,After Stopword Removal
548,program beasiswa di uin sunan gunung djati ban...,program beasiswa uin sunan gunung djati bandun...
549,apakah beasiswa di uin sunan gunung djati band...,beasiswa uin sunan gunung djati bandung tawar
550,program beasiswa di uin sunan gunung djati ban...,program beasiswa uin sunan gunung djati bandun...
551,apakah program beasiswa yang ada di uin sunan ...,program beasiswa ada uin sunan gunung djati ba...
552,program beasiswa yang tawar oleh uin sunan gun...,program beasiswa tawar uin sunan gunung djati ...
553,apakah program beasiswa sedia oleh uin sunan g...,program beasiswa sedia uin sunan gunung djati ...
554,program beasiswa di uin sunan gunung djati ban...,program beasiswa uin sunan gunung djati bandun...
555,apakah beasiswa di uin sunan gunung djati band...,beasiswa uin sunan gunung djati bandung ada
556,program beasiswa di uin sunan gunung djati ban...,program beasiswa uin sunan gunung djati bandun...
557,apakah beasiswa di uin sunan gunung djati band...,beasiswa uin sunan gunung djati bandung beri


### Filtering (Drop Duplicate Data)

In [42]:
to_be_filtered = pd.DataFrame({
    "pattern": hasil_stopword_removal,
    "tag": question_data['tag']
})
to_be_filtered.head(10)
hasil_filtering = to_be_filtered

hasil_filtering.to_csv('data/preprocessed-data-v2.csv', index=False)

print('Filtered data', hasil_filtering.shape)
hasil_filtering.tail(10)

Filtered data (558, 2)


,pattern,tag
548,program beasiswa uin sunan gunung djati bandun...,beasiswa
549,beasiswa uin sunan gunung djati bandung tawar,beasiswa
550,program beasiswa uin sunan gunung djati bandun...,beasiswa
551,program beasiswa ada uin sunan gunung djati ba...,beasiswa
552,program beasiswa tawar uin sunan gunung djati ...,beasiswa
553,program beasiswa sedia uin sunan gunung djati ...,beasiswa
554,program beasiswa uin sunan gunung djati bandun...,beasiswa
555,beasiswa uin sunan gunung djati bandung ada,beasiswa
556,program beasiswa uin sunan gunung djati bandun...,beasiswa
557,beasiswa uin sunan gunung djati bandung beri,beasiswa


### Preprocessed Data

In [19]:
dataset = pd.read_csv('data/preprocessed-data-v2.csv')
print("Total data: ", dataset.shape)
dataset.tail(10)

Total data:  (404, 2)


,pattern,tag
394,program beasiswa uin sunan gunung djati bandun...,beasiswa
395,program beasiswa beri uin sunan gunung djati b...,beasiswa
396,program beasiswa uin sunan gunung djati bandun...,beasiswa
397,beasiswa uin sunan gunung djati bandung tawar,beasiswa
398,program beasiswa uin sunan gunung djati bandun...,beasiswa
399,program beasiswa ada uin sunan gunung djati ba...,beasiswa
400,program beasiswa tawar uin sunan gunung djati ...,beasiswa
401,program beasiswa uin sunan gunung djati bandun...,beasiswa
402,beasiswa uin sunan gunung djati bandung ada,beasiswa
403,beasiswa uin sunan gunung djati bandung beri,beasiswa


## Data Labelling

In [20]:
# # Defining labels
# label_list = dataset['tag'].drop_duplicates()
# LABELS = label_list.tolist()
# len(LABELS), LABELS

In [21]:
# # Give 0 as initial value
# labelled_data = dataset.copy()
# labelled_data[LABELS] = 0
# labels = pd.Series(label_list)

# for i, row in labelled_data.iterrows():
#     tag = row['tag'].split(';')
#     labelled_data.loc[i, tag] = 1
# labelled_data.tail(1)
# # labelled_data[LABELS].sum()

## Working Model

In [22]:
# from sentence_transformers import SentenceTransformer
# sentences = []
# for sentence in dataset['pattern']:
#     sentences.append(sentence)

# model = SentenceTransformer('firqaaa/indo-sentence-bert-base')
# embeddings = model.encode(sentences)

In [23]:
# def low_confident(indexes):
#     tags = []
#     data = dataset.to_numpy().tolist()
#     pos = 0
#     for tag in data:
#         for id in indexes:
#             if(id == pos):
#                 tags.append(tag)
#         pos += 1
#     tags = pd.DataFrame(tags)
#     tags = tags.drop_duplicates()
#     return tags

In [24]:
# asked_quetion = 'Informasi seputar biaya UKT di UIN Sunan Gunung Djati Bandung'
# encoded_question = model.encode(asked_quetion)
# result = np.array(model.similarity(encoded_question, embeddings))
# highestIndex = np.argmax(result)
# highestVal = np.max(result)
# tag = dataset.loc[highestIndex]['tag']

# print
# highestIndex, highestVal, tag

In [25]:
# # get confidents higher than 0.5
# confidents = []
# indexes = []
# pos = 0

# for item in result:
#     for value in item:
#         if(value >= 0.8):
#             confidents.append(value)
#             indexes.append(pos)
#         elif(value < 0.8):
#             recommended_tags = low_confident(indexes)
#             recommended_tags
#         pos += 1
# confidents, indexes

### Manual

In [26]:
# from transformers import AutoTokenizer, AutoModel
# import torch


# #Mean Pooling - Take attention mask into account for correct averaging
# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# # Sentences we want sentence embeddings for
# sentences = ["Ibukota Perancis adalah Paris", 
#              "Menara Eifel terletak di Paris, Perancis", 
#              "Pizza adalah makanan khas Italia", 
#              "Saya kuliah di Carneige Mellon University"]


# # Load model from HuggingFace Hub
# tokenizer = AutoTokenizer.from_pretrained('firqaaa/indo-sentence-bert-base')
# model = AutoModel.from_pretrained('firqaaa/indo-sentence-bert-base')

# # Tokenize sentences
# encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# # Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)

# # Perform pooling. In this case, mean pooling.
# sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# print("Sentence embeddings:")
# print(sentence_embeddings)

## Fine-tuned Model

In [27]:
# %pip install -U "sentence-transformers[train]" " transformers[torch]" accelerate datasets

In [28]:
# # Import libraries
# from datasets import load_dataset
# from sentence_transformers import (
#     SentenceTransformer,
#     SentenceTransformerTrainer,
#     SentenceTransformerTrainingArguments
# )
# from sentence_transformers.losses import CoSENTLoss
# from sentence_transformers.training_args import BatchSamplers
# from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

In [29]:
# # Define model
# model = SentenceTransformer("firqaaa/indo-sentence-bert-base")

In [30]:
# # Define datasets
# train_dataset = load_dataset("csv", data_files="data/preprocessed-data.csv")
# test_dataset = load_dataset("csv", data_files="data/dataset-question.csv")
# eval_dataset = load_dataset("csv", data_files="data/dataset-eval.csv")
# train_dataset, test_dataset

In [31]:
# # Define loss function (CoSENTLoss | Cosine Sentence Loss -> Returning float similarity score)
# loss = CoSENTLoss(model)

In [32]:
# # Specify training args
# args = SentenceTransformerTrainingArguments(
#     output_dir="fine-tuned/sbert-fine-tuned-chatPMB",
#     num_train_epochs=50,
#     per_device_train_batch_size=24,
#     per_device_eval_batch_size=24,
#     learning_rate=2e-5,
#     warmup_ratio=0.1,
#     batch_sampler=BatchSamplers.NO_DUPLICATES,
#     eval_strategy="steps",
#     eval_steps=100,
#     save_strategy="steps",
#     save_steps=100,
#     save_total_limit=2,
#     logging_steps=100,
# )

In [33]:
# # Creating eval dataset
# train_patterns = pd.read_csv('data/preprocessed-data.csv')['pattern']
# test_patterns = pd.read_csv('data/dataset-question.csv')['pattern']
# test_patterns = test_patterns.iloc[:-12]
# train_patterns, test_patterns

In [34]:
# embed_train = model.encode(train_patterns)
# embed_test = model.encode(test_patterns)
# # embed_test, embed_train

In [35]:
# scores = []
# temp = 0
# result = np.array(model.similarity(embed_train, embed_test))
# # result
# for val in result:
#     for data in val:
#         if(temp<data):
#             temp=data
#     scores.append(temp)
# dev = pd.DataFrame({
#     "train_patterns": train_patterns,
#     "test_patterns": test_patterns,
#     "score": scores
# })
# # dev.to_csv('data/dataset-eval.csv')
# dev

In [36]:
# %pip install seaborn matplotlib

In [37]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# plt.figure(figsize=(15, 15))
# sns.heatmap(
#     result,
#     annot=True,
#     fmt="d",
#     cmap="Blues",
#     xticklabels=train_patterns,
#     yticklabels=test_patterns,
# )
# plt.xlabel("Predicted labels")
# plt.ylabel("True labels")
# plt.title("Training Data")
# plt.show()

In [38]:
# # Create evaluator & evaluate the base model
# dev_evaluator = EmbeddingSimilarityEvaluator(
#     sentences1=dev["train_patterns"],
#     sentences2=dev["test_patterns"],
#     scores=dev["score"],
#     main_similarity=SimilarityFunction.COSINE,
#     name="sts-dev",
# )
# dev_evaluator(model)

In [39]:
# trainer = SentenceTransformerTrainer(
#     model=model,
#     args=args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     loss=loss,
#     evaluator=dev_evaluator,
# )
# trainer.train()

In [40]:
# model.save_pretrained("fine-tuned/models/chatPMB-SBERT-pretrained")

In [41]:
# test_evaluator = EmbeddingSimilarityEvaluator(
#     sentences1=dev["train_patterns"],
#     sentences2=dev["test_patterns"],
#     scores=dev["score"],
#     main_similarity=SimilarityFunction.COSINE,
#     name="sts-dev",
# )
# test_evaluator(model)